# testing

## Launching

In [31]:
!run test.cmd

'run' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [ ]:
!bash test.sh

## Global Setting

In [23]:
tester = "Axel"
network_testing = True
address = "127.0.0.1:5000"

# Global Setting reflect to settings
if tester == "Axel":
    if not network_testing:
        address = '100.86.126.26:5000'
    if network_testing:
        address = '172.20.10.4:6000'


In [23]:
# Reloading the lib
import importlib
import crypto
importlib.reload(crypto)

<module 'crypto' from 'f:\\uniIssue\\Comp3221\\A3\\A3\\crypto.py'>

## Testing

In [10]:
import socket
import network
import json

def send_transaction(address, msg: dict):
    '''
    msg is a json object being sent
    '''
    msg_str = json.dumps(msg)
    msg_bytes = msg_str.encode('utf8')
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host, port = address.split(':')
    client_socket.connect((host, int(port)))

    try:
        # Send the transaction
        network.send_prefixed(client_socket, msg_bytes)

        # Wait for the response
        response = network.recv_prefixed(client_socket)
        response_json = json.loads(response.decode('utf8'))
        print(f"Received response: {response_json}")

    finally:
        # Close the socket connection regardless of success or failure
        client_socket.close()

In [28]:
# Directly send
message = {
    "type": "transaction",
    "payload": {
        "sender": "a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b",
        "message": "hello",
        "nonce": 0,
        "signature": "142e395895e0bf4e4a3a7c3aabf2f59d80c517d24bb2d98a1a24384bc7cb29c9d593ce3063c5dd4f12ae9393f3345174485c052d0f5e87c082f286fd60c7fd0c"
    }
}

send_transaction(address, message)

Received response: {'response': False}


In [11]:
# Generate keypair (then send)
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import ed25519
import json
import crypto

# Generate RSA key pair
private_key, public_key = crypto.generate_keypair_bytes()

# Export the public key
public_key_hex = crypto.publickey_bytes_to_hex(public_key)

In [26]:
payload = {
    "sender": public_key_hex,
    "message": "abaaba",
    "nonce": 1,
}

message_wo_sig = json.dumps({
        "sender": payload["sender"],
        "message": payload["message"],
        "nonce": payload["nonce"]},
        sort_keys=True)
message_wo_sig_bytes = message_wo_sig.encode()

# Generate signature
signature = crypto.encrypt_with_privatekey(message_wo_sig_bytes, private_key)

payload['signature'] = signature.hex()

message = {
    "type": "transaction",
    "payload": payload
}

print(f"The message sent is {message}")
print(f"Send to {address}")

send_transaction(address, message)

The message sent is {'type': 'transaction', 'payload': {'sender': '6a927a3a58599cefb4aa316c8a3e8e387206d239aec0dd8dba306404fa68bf92', 'message': 'abaaba', 'nonce': 1, 'signature': '72488e7ffe36568321a04ade2e542957edb9238f64afe44022de7845ba939b7d4226a983776158c1443d1a174f646a849ac86bde585cb50b4bd0506295ca8e02'}}
Send to 172.20.10.4:6000
Received response: {'response': True}


## Debugging

In [43]:
# For debugging: verify the signature is correct
signature_hex = payload["signature"]
signature_bytes = bytes.fromhex(signature_hex)

public_key_bytes = bytes.fromhex(payload["sender"])
public_key = ed25519.Ed25519PublicKey.from_public_bytes(public_key_bytes)

try:
    public_key.verify(signature_bytes, message_wo_sig_bytes)
    print("The signature is correct")
except Exception as e:
    print("the signature is not corresponding to the message")

The signature is correct


# After each testing

In [16]:
device_used = "mac"

In [30]:
# Delete debug files
import os
import glob

files = glob.glob('debug_for_*.txt')
for f in files:
    os.remove(f)

In [ ]:
!netstat -aon | findstr :5000

In [ ]:
!taskkill /PID 29216 /F